In [ ]:
sc.stop()

In [1]:
import os
import time

import functions
from mappings import *

import numpy as np
import pandas as pd

from itertools import chain
from functools import reduce
from datetime import datetime
from collections import defaultdict

from pyspark.sql import Window, SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml.classification import LinearSVC, LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from elasticsearch import Elasticsearch

import matplotlib.pyplot as plt

%matplotlib inline

# Initialize Spark

We will create our Spark program using **SparkSession** ...

In [2]:
appName = "my-tfm"

ss = SparkSession.builder.appName(appName).config("spark.some.config.option", "4g").getOrCreate()
print(ss)
sc = ss.sparkContext
print(sc)

<SparkContext master=yarn appName=my-tfm>


# Loading

We will load the data previously downloaded from its source:

* **Historical data:** http://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008

* **Prediction data:** https://www.kaggle.com/kumargh/pimaindiansdiabetescsv

* **Analysis (time-series):** http://archive.ics.uci.edu/ml/datasets/Diabetes

In [45]:
# Load first dataset (historical data)
hist_data = functions.load_data(ss, "diabetic_data.csv")

22-08-2020 23:55:27 - Loading diabetic_data.csv file....
Loaded 101766 rows and 50 columns
+------------+-----------+---------+------+------+------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|     race|gender|   age|weight|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_out

In [46]:
# Load second dataset (prediction data)
col_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',\
            'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Class']
pred_data = functions.load_data(ss, "pima-indians-diabetes.csv", hdr=False, col_names=col_names)

22-08-2020 23:55:31 - Loading pima-indians-diabetes.csv file....
Loaded 768 rows and 9 columns
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Class|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|    1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
only showing top 1 row

22-08-2020 23:55:32 - Load completed. This operation has taken 0.65 s


In [47]:
# Load third dataset (analysis data)
an_data = functions.load_ts(sc, 70, 'data') # the function load_ts already imputes missing values when loading the file

22-08-2020 23:55:32 - Loading data-01 file....
Loaded 943 rows and 5 columns
22-08-2020 23:55:40 - Loading data-02 file....
Loaded 761 rows and 5 columns
22-08-2020 23:55:41 - Loading data-03 file....
Loaded 300 rows and 5 columns
22-08-2020 23:55:42 - Loading data-04 file....
Loaded 300 rows and 5 columns
22-08-2020 23:55:42 - Loading data-05 file....
Loaded 300 rows and 5 columns
22-08-2020 23:55:43 - Loading data-06 file....
Loaded 149 rows and 5 columns
22-08-2020 23:55:43 - Loading data-07 file....
Loaded 242 rows and 5 columns
22-08-2020 23:55:44 - Loading data-08 file....
Loaded 177 rows and 5 columns
22-08-2020 23:55:44 - Loading data-09 file....
Loaded 206 rows and 5 columns
22-08-2020 23:55:45 - Loading data-10 file....
Loaded 247 rows and 5 columns
22-08-2020 23:55:45 - Loading data-11 file....
Loaded 236 rows and 5 columns
22-08-2020 23:55:46 - Loading data-12 file....
Loaded 300 rows and 5 columns
22-08-2020 23:55:46 - Loading data-13 file....
Loaded 300 rows and 5 columns

# Preprocessing

### Historical data

Remove the rows and columns with more than 30% of missing values and replace the rest with the mode of the column.

In [49]:
hist_data = functions.impute_na(hist_data)


22-08-2020 23:57:20 - Working with missing values...
Removing rows with more than 30% of missing data...
0 rows removed

Showing columns with missing values: 
                   count  density
race                2273     0.02
weight             98569     0.97
payer_code         40256     0.40
medical_specialty  49949     0.49
diag_1                21     0.00
diag_2               358     0.00
diag_3              1423     0.01

Removing rows with more than 30% of missing data (weight, payer_code, medical_specialty) ...

23-08-2020 00:08:24 - Columns removed. 
New shape: (101766 rows x 47 columns)

23-08-2020 00:08:25 - Replacing missing values...
Replacing NAs values of attribute race by value Caucasian
Replacing NAs values of attribute diag_1 by value 428
Replacing NAs values of attribute diag_2 by value 276
Replacing NAs values of attribute diag_3 by value 250

23-08-2020 00:09:12 - Missing values removed.  This operation has taken 711.97 s


Cast IDs with numbers to String.

In [50]:
hist_data = hist_data.withColumn("encounter_id",hist_data["encounter_id"].cast(StringType())) \
                    .withColumn("discharge_disposition_id",hist_data["discharge_disposition_id"].cast(StringType())) \
                    .withColumn("admission_source_id",hist_data["admission_source_id"].cast(StringType())) \
                    .withColumn("admission_type_id",hist_data["admission_type_id"].cast(StringType())) \
                    .withColumn("patient_nbr",hist_data["patient_nbr"].cast(StringType()))

hist_data.printSchema()

root
 |-- encounter_id: string (nullable = true)
 |-- patient_nbr: string (nullable = true)
 |-- race: string (nullable = false)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- admission_type_id: string (nullable = true)
 |-- discharge_disposition_id: string (nullable = true)
 |-- admission_source_id: string (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_outpatient: integer (nullable = true)
 |-- number_emergency: integer (nullable = true)
 |-- number_inpatient: integer (nullable = true)
 |-- diag_1: string (nullable = false)
 |-- diag_2: string (nullable = false)
 |-- diag_3: string (nullable = false)
 |-- number_diagnoses: integer (nullable = true)
 |-- max_glu_serum: string (nullable = true)
 |-- A1Cresult: string (nullable = true)
 |-- metformin: string (nullable = true)
 |-- r

Explore the data

In [48]:
data_types = defaultdict(list)
for entry in hist_data.schema.fields:
     data_types[str(entry.dataType)].append(entry.name)
        
categorical_variables = len(data_types["StringType"])
print("Number of categorical variables: {}\t Percentage: {:.2f}%". format(categorical_variables, \
                                                                      100*(categorical_variables/len(hist_data.columns))))
numerical_variables = len(data_types["IntegerType"])
print("Number of numerical variables: {}\t Percentage: {:.2f}%". format(numerical_variables, \
                                                                    100*(numerical_variables/len(hist_data.columns))))

Number of categorical variables: 37	 Percentage: 74.00%
Number of numerical variables: 13	 Percentage: 26.00%


In [ ]:
plt.hist(hist_data.select("metformin-pioglitazone").toPandas())

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50508)
Traceback (most recent call last):
  File "/home/marta/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/marta/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/marta/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/marta/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 270, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 242, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 246, in accum_updates
    num_updates = read_int(self

Filter the columns to be used in the machine learning model in a new variable. 

In [8]:
columns_to_drop = ['encounter_id', 'race', 'age', 'payer_code', 'medical_specialty']
hist_data_aux = hist_data.drop(*columns_to_drop)

hist_data_aux = hist_data_aux.select(['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 
                                 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 
                                  'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
                                 'glipizide-metformin', 'glimepiride-pioglitazone', 'readmitted'])

Map the necessary variables

In [9]:
## New dataset: transform to binary class
mapping_expr = create_map([lit(x) for x in chain(*readmitted_mapping.items())])
hist_data_aux = hist_data_aux.withColumn("readmitted", mapping_expr.getItem(col("readmitted"))) 

## Final dataset: admission type
mapping_expr = create_map([lit(x) for x in chain(*admission_type_id_mapping.items())])
hist_data = hist_data.withColumn("admission_type_id", mapping_expr.getItem(col("admission_type_id")))

## Final dataset: 
mapping_expr = create_map([lit(x) for x in chain(*discharge_disposition_id_mapping.items())])
hist_data = hist_data.withColumn("discharge_disposition_id", mapping_expr.getItem(col("discharge_disposition_id")))

## Final dataset: 
mapping_expr = create_map([lit(x) for x in chain(*admission_source_id_mapping.items())])
hist_data = hist_data.withColumn("admission_source_id", mapping_expr.getItem(col("admission_source_id")))

## Final dataset: 
mapping_expr = create_map([lit(x) for x in chain(*meds_mapping.items())])
hist_data = hist_data.withColumn("metformin", mapping_expr.getItem(col("metformin"))) \
                    .withColumn("repaglinide", mapping_expr.getItem(col("repaglinide"))) \
                    .withColumn("nateglinide", mapping_expr.getItem(col("nateglinide"))) \
                    .withColumn("chlorpropamide", mapping_expr.getItem(col("chlorpropamide"))) \
                    .withColumn("glimepiride", mapping_expr.getItem(col("glimepiride"))) \
                    .withColumn("acetohexamide", mapping_expr.getItem(col("acetohexamide"))) \
                    .withColumn("glipizide", mapping_expr.getItem(col("glipizide"))) \
                    .withColumn("glyburide", mapping_expr.getItem(col("glyburide"))) \
                    .withColumn("tolbutamide", mapping_expr.getItem(col("tolbutamide"))) \
                    .withColumn("pioglitazone", mapping_expr.getItem(col("pioglitazone"))) \
                    .withColumn("rosiglitazone", mapping_expr.getItem(col("rosiglitazone"))) \
                    .withColumn("acarbose", mapping_expr.getItem(col("acarbose"))) \
                    .withColumn("miglitol", mapping_expr.getItem(col("miglitol"))) \
                    .withColumn("troglitazone", mapping_expr.getItem(col("troglitazone"))) \
                    .withColumn("tolazamide", mapping_expr.getItem(col("tolazamide"))) \
                    .withColumn("examide", mapping_expr.getItem(col("examide"))) \
                    .withColumn("citoglipton", mapping_expr.getItem(col("citoglipton"))) \
                    .withColumn("insulin", mapping_expr.getItem(col("insulin"))) \
                    .withColumn("glyburide-metformin", mapping_expr.getItem(col("glyburide-metformin"))) \
                    .withColumn("glipizide-metformin", mapping_expr.getItem(col("glipizide-metformin"))) \
                    .withColumn("glimepiride-pioglitazone", mapping_expr.getItem(col("glimepiride-pioglitazone"))) \
                    .withColumn("metformin-rosiglitazone", mapping_expr.getItem(col("metformin-rosiglitazone"))) \
                    .withColumn("metformin-pioglitazone", mapping_expr.getItem(col("metformin-pioglitazone")))

## Final dataset: 
mapping_expr = create_map([lit(x) for x in chain(*change_mapping.items())])
hist_data = hist_data.withColumn("change", mapping_expr.getItem(col("change")))

## Final dataset: 
mapping_expr = create_map([lit(x) for x in chain(*diabetesMed_mapping.items())])
hist_data = hist_data.withColumn("diabetesMed", mapping_expr.getItem(col("diabetesMed")))

## Final dataset: 
mapping_expr = create_map([lit(x) for x in chain(*readmitted_mapping.items())])
hist_data = hist_data.withColumn("readmitted", mapping_expr.getItem(col("readmitted"))) \
                        .withColumnRenamed("readmitted", "readmitted_trunc")

## Final dataset: diagnostics 
hist_data = hist_data.withColumn("diag_1", translate(col("diag_1"), ".", "")) \
                     .withColumn("diag_2", translate(col("diag_2"), ".", "")) \
                     .withColumn("diag_3", translate(col("diag_3"), ".", ""))

df = ss.read.load("tfm-marta/data/CMS30_DESC_LONG_DX.csv", format="csv", sep=";", inferSchema=True)
diag_mapping = df.toPandas().set_index('_c0').to_dict()['_c1']
diag_mapping

mapping_expr = create_map([lit(x) for x in chain(*diag_mapping.items())])
hist_data = hist_data.withColumn("diag_1", mapping_expr.getItem(col("diag_1"))) \
                     .withColumn("diag_2", mapping_expr.getItem(col("diag_2"))) \
                     .withColumn("diag_3", mapping_expr.getItem(col("diag_3")))

## Final dataset: add new column: date
hist_data = hist_data.withColumn('date', lit(datetime.now().strftime("%d-%m-%Y")))
hist_data = hist_data.withColumn('date', to_timestamp(unix_timestamp(hist_data.date, 'MM-dd-yyyy')))

Transform categorical variables with more than 1 category using One-hot-encoding

In [11]:
## count number of categories of each categorical variable
data_types = defaultdict(list)
for entry in hist_data_aux.schema.fields:
     data_types[str(entry.dataType)].append(entry.name)

counts_summary = hist_data_aux.agg(*[countDistinct(c).alias(c) for c in data_types["StringType"]])
counts_summary = counts_summary.toPandas()

counts = pd.Series(counts_summary.values.ravel())
counts.index = counts_summary.columns

sorted_vars = counts.sort_values(ascending = False) # number of categories of each variable
ignore = list((sorted_vars[sorted_vars <=1]).index) # variables with less than 1 category
# We'll ignore the categorical variables with only 1 category

strings_used = [var for var in data_types["StringType"] if var not in ignore]
stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used]
stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used]

ppl = Pipeline(stages= stage_string + stage_one_hot)
hist_data_trunc = ppl.fit(hist_data_aux).transform(hist_data_aux)

Combine all the features into a single vector for each observation

In [12]:
features = data_types["IntegerType"] + [var + "_one_hot" for var in strings_used]
hist_data_trunc = VectorAssembler(inputCols = [x for x in features if x not in 'readmitted'], outputCol= "features")\
                .transform(hist_data_trunc).select(['readmitted', 'features'])

Split data into training, validation and test sets

In [13]:
## Split data into train, validation and test
train_histC_trunc, test_hist_trunc = functions.df_split(hist_data_trunc, 0.7)
train_hist_trunc, val_hist_trunc = functions.df_split(train_histC_trunc, 0.7)

## Split data into X and Y coordinates
train_hist_trunc, val_hist_trunc =  functions.df_split(train_histC_trunc, 0.7)

print("train_hist: {0} x {1}\t val_hist: {2} x {3}\t test_hist: {4} x {5}"\
      .format(train_hist_trunc.count(), len(train_hist_trunc.columns),
              val_hist_trunc.count(), len(val_hist_trunc.columns),
              test_hist_trunc.count(), len(test_hist_trunc.columns)))

train_hist: 49690 x 2	 val_hist: 21370 x 2	 test_hist: 30706 x 2


### Prediction data

Remove the rows and columns with more than 30% of missing values and replace the rest with the mode of the column.

In [14]:
pred_data = functions.impute_na(pred_data)
pred_data.printSchema()


22-08-2020 23:12:57 - Working with missing values...
Removing rows with more than 30% of missing data...
0 rows removed

Showing columns with missing values: 
Empty DataFrame
Columns: [count, density]
Index: []

Removing rows with more than 30% of missing data () ...

22-08-2020 23:13:02 - Columns removed. 
New shape: (768 rows x 9 columns)

22-08-2020 23:13:02 - Replacing missing values...

22-08-2020 23:13:03 - Missing values removed.  This operation has taken 5.98 s
root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Class: integer (nullable = true)



Split the dataset into validation (21%), training (49%) and test (30%)

In [15]:
## Split data into train, validation and test
train_predC, test_pred = functions.df_split(pred_data, 0.7)
train_pred, val_pred = functions.df_split(train_predC, 0.7)

## Split data into X and Y coordinates
Xtrain_pred, ytrain_pred = train_pred.select(train_pred.columns[0:len(train_pred.columns)-1]), train_pred.select('Class')
Xval_pred, yval_pred =  val_pred.select(val_pred.columns[0:len(val_pred.columns)-1]), val_pred.select('Class')
Xtest_pred, ytest_pred = test_pred.select(test_pred.columns[0:len(test_pred.columns)-1]), test_pred.select('Class')

print("Xtrain_pred: {0} x {1}\t ytrain_pred: {2} x {3}".format(Xtrain_pred.count(), len(Xtrain_pred.columns),
                                                              ytrain_pred.count(), len(ytrain_pred.columns)))
print("Xval_pred: {0} x {1}\t yval_pred: {2} x {3}".format(Xval_pred.count(), len(Xval_pred.columns),
                                                              yval_pred.count(), len(yval_pred.columns)))
print("Xtest_pred: {0} x {1}\t ytest_pred: {2} x {3}".format(Xtest_pred.count(), len(Xtest_pred.columns),
                                                              ytest_pred.count(), len(ytest_pred.columns)))

Xtrain_pred: 368 x 8	 ytrain_pred: 368 x 1
Xval_pred: 165 x 8	 yval_pred: 165 x 1
Xtest_pred: 235 x 8	 ytest_pred: 235 x 1


Combine all the features into a single vector for each observation

In [16]:
# Put together all the input attributes.
assembler = VectorAssembler(inputCols=[x for x in train_predC.columns if x not in 'Class'], outputCol='features')

train_predC_trunc = assembler.transform(train_predC).select(['Class', 'features'])
train_pred_trunc = assembler.transform(train_pred).select(['Class', 'features'])
val_pred_trunc = assembler.transform(val_pred).select(['Class', 'features'])
test_pred_trunc = assembler.transform(test_pred).select(['Class', 'features'])

+-----+--------------------+
|Class|            features|
+-----+--------------------+
|    0|[0.0,86.0,68.0,32...|
|    0|[0.0,91.0,80.0,0....|
|    0|[0.0,95.0,64.0,39...|
+-----+--------------------+
only showing top 3 rows

None


Add columns: date and patient number

In [17]:
pred_data = pred_data.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
pred_data = pred_data.withColumn("patient_nbr", 50125+row_number().over(w)).drop("new_column")
pred_data = pred_data.withColumn("patient_nbr",pred_data["patient_nbr"].cast(StringType()))
pred_data = pred_data.withColumn('date', lit(datetime.now().strftime("%d-%m-%Y")))
pred_data = pred_data.withColumn('date', to_timestamp(unix_timestamp(pred_data.date, 'MM-dd-yyyy')))

###  Analytical data

In [ ]:
an_data.printSchema()

In [ ]:
an_data = an_data.withColumn("value",an_data["value"].cast(IntegerType())) 


mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])
an_data = an_data.withColumn("code", mapping_expr.getItem(col("code"))) \
                 .withColumn("patient_nbr",an_data["patient_nbr"].cast(StringType())) \
                 .withColumn('date', an_data.date + expr('INTERVAL 29 years'))
an_data.printSchema()
an_data.show()

# Analyzing and predicting the data

### Historical data

Explore the data

In [ ]:
df.select("metformin").distinct().rdd.map(lambda r: r[0]).collect()

In [21]:
lr = LogisticRegression(maxIter=1000, regParam=0, elasticNetParam=0.8, featuresCol="features", labelCol="readmitted")
model = lr.fit(train_hist_trunc)

evaluator = BinaryClassificationEvaluator(labelCol='readmitted')

y_train_hist = model.transform(train_hist_trunc)
train_accuracy = evaluator.evaluate(y_train_hist)
print("Training accuracy: {:.2f}".format(train_accuracy))

y_val_hist = model.transform(val_hist_trunc)
val_accuracy = evaluator.evaluate(y_val_hist)
print("Validation accuracy: {:.2f}".format(val_accuracy))

y_test = model.transform(test_hist_trunc)
test_accuracy = evaluator.evaluate(y_test)
print("Test accuracy: {:.2f}".format(test_accuracy))

Training accuracy: 0.55
Validation accuracy: 0.55
Test accuracy: 0.55


Add column with the result

In [22]:
hist_data = hist_data.withColumn("Accuracy", lit(test_accuracy))

hist_data.printSchema()

root
 |-- encounter_id: string (nullable = true)
 |-- patient_nbr: string (nullable = true)
 |-- race: string (nullable = false)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- admission_type_id: string (nullable = true)
 |-- discharge_disposition_id: string (nullable = true)
 |-- admission_source_id: string (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_outpatient: integer (nullable = true)
 |-- number_emergency: integer (nullable = true)
 |-- number_inpatient: integer (nullable = true)
 |-- diag_1: string (nullable = true)
 |-- diag_2: string (nullable = true)
 |-- diag_3: string (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- max_glu_serum: string (nullable = true)
 |-- A1Cresult: string (nullable = true)
 |-- metformin: integer (nullable = true)
 |-- rep

### Prediction data

blablablablabla

In [ ]:
pred_data.describe().show()

In [24]:
model = LinearSVC(labelCol='Class')
paramGrid = ParamGridBuilder() \
        .addGrid(model.regParam, [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]) \
        .build()
evaluator = BinaryClassificationEvaluator(labelCol='Class')
crossval = CrossValidator(estimator=model, estimatorParamMaps=paramGrid, evaluator=evaluator, parallelism=2, numFolds=5)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train_pred_trunc)
print("Regularization parameter: {}".format(cvModel.bestModel._java_obj.getRegParam()))

y_train_pred = cvModel.transform(train_pred_trunc)
train_accuracy = evaluator.evaluate(y_train_pred)
print("Training accuracy: {:.2f}".format(train_accuracy))

y_val_pred = cvModel.transform(val_pred_trunc)
val_accuracy = evaluator.evaluate(y_val_pred)
print("Validation accuracy: {:.2f}".format(val_accuracy))

### Final model
model = LinearSVC(labelCol='Class', regParam=cvModel.bestModel._java_obj.getRegParam()).fit(train_predC_trunc)
y_test = model.transform(test_pred_trunc)
test_accuracy = evaluator.evaluate(y_test)
print("Test accuracy: {:.2f}".format(test_accuracy))

Regularization parameter: 0.001
Training accuracy: 0.85
Validation accuracy: 0.84
Test accuracy: 0.80


In [25]:
pred_data = pred_data.withColumn("Accuracy", lit(test_accuracy))
pred_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+-----------+----+------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Class|patient_nbr|date|          Accuracy|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+-----------+----+------------------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|    1|      50126|null|0.7998110614908964|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|    0|      50127|null|0.7998110614908964|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|    1|      50128|null|0.7998110614908964|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|    0|      50129|null|0.7998110614908964|
|          0|    137|           40|           35|    168|43.1|

### Analysis data

blablablablabla

# Indexing data into elasticsearch

In [ ]:
myHost = "192.168.146.131"

es = Elasticsearch([{"host": myHost,"port":9200}]) # Connect to the elastic cluster
print(es)

def uploadToES(indexName, indexBody, docType=None, requestBody=None, verbose=True):
    if (es.indices.exists(indexName)==False): 
        es.indices.create(index=indexName, body=requestBody)
        if(verbose): print("creating {} index...".format(indexName))
    es.index(index=indexName,doc_type=docType, body=indexBody) # Store the document in Elasticsearch
    if(verbose): print("adding document {} to {} index...".format(indexBody, indexName))

In [ ]:
aux=hist_data.limit(200).toPandas()
for i in range(0, aux.shape[0]):
    body=aux.iloc[i].to_json(orient="index", date_format='iso')
    uploadToES("tfm-marta-historical-data", body)


aux=pred_data.toPandas()
for i in range(0, aux.shape[0]):
    body=aux.iloc[i].to_json(orient="index", date_format='iso')
    uploadToES("tfm-marta-prediction-data", body)
    
# an_requestBody = {
#     "mappings": {
#         "properties": {
#             "date": {"type": "date", "format":"yyyy-MM-dd HH:mm || yyyy-MM-dd || epoch_millis", "ignore_malformed":True},
#             "code": {"type": "keyword"},
#             "value":{"type": "long"}
#         }
#     }
# }

aux=an_data.toPandas()    
for j in range(0, aux.shape[0]):   
    body=aux.iloc[j].to_json(orient="index", date_format='iso')
    uploadToES("tfm-marta-analysis-data", body)